In [1]:
import numpy as np

In [12]:
import ollama
import pandas as pd
import numpy as np
from datasets import load_dataset
from sklearn.metrics import accuracy_score
from tqdm import tqdm
import os


In [13]:
os.environ["HF_HUB_DISABLE_SYMLINKS_WARNING"] = "1"

In [14]:

dataset = load_dataset("ag_news")

test_texts = dataset["test"]["text"]
test_labels = dataset["test"]["label"]  

# label mapping 
label_mapping = {
    0: "World",
    1: "Sports",
    2: "Business",
    3: "Science/Technology"
}

In [34]:
test_texts[27], test_labels[27]

("Tougher rules won't soften Law's game FOXBOROUGH -- Looking at his ridiculously developed upper body, with huge biceps and hardly an ounce of fat, it's easy to see why Ty Law, arguably the best cornerback in football, chooses physical play over finesse. That's not to imply that he's lacking a finesse component, because he can shut down his side of the field much as Deion Sanders ...",
 1)

In [50]:

def classify_text_ollama(text):
    prompt = f"Answer with one word. Classify the following news article into one of these categories which fits best: World, Sports, Business, or Science/Technology.\n\nArticle: {text}\n\nCategory:"

    output = ollama.chat(model="llama3.2", messages=[{"role": "user", "content": prompt}])

    response = output["message"]["content"].strip()    
    #print(f"response: {response}")   
    
    for idx, category in label_mapping.items():
        if category.lower() in response.lower():
            return idx  

    print("no match found")
    return -1  

In [51]:

predicted_labels = []
for text in tqdm(test_texts):  
    predicted_labels.append(classify_text_ollama(text))

 27%|██▋       | 2087/7600 [01:29<03:40, 24.98it/s]

no match found


 28%|██▊       | 2105/7600 [01:29<03:52, 23.68it/s]

no match found


 32%|███▏      | 2435/7600 [01:43<03:46, 22.83it/s]

...

 72%|███████▏  | 5495/7600 [03:51<01:34, 22.31it/s]

no match found


 92%|█████████▏| 7002/7600 [04:54<00:26, 22.86it/s]

no match found


100%|██████████| 7600/7600 [05:19<00:00, 23.80it/s]


In [55]:
predicted_labels = np.array(predicted_labels)
true_labels = np.array(test_labels[:])  

## Compute Accuracy

In [56]:

valid_indices = predicted_labels != -1  
accuracy = accuracy_score(true_labels[valid_indices], predicted_labels[valid_indices])

print(f"\nClassification Accuracy: {accuracy:.2%} on {len(valid_indices)} samples")


Classification Accuracy: 58.87% on 7600 samples


In [57]:

# sample classifications
for i in range(5):
    print(f"\nNews: {test_texts[i]}")
    print(f"True Label: {label_mapping[test_labels[i]]}")
    print(f"Predicted Label: {label_mapping.get(predicted_labels[i], 'Unknown')}")


News: Fears for T N pension after talks Unions representing workers at Turner   Newall say they are 'disappointed' after talks with stricken parent firm Federal Mogul.
True Label: Business
Predicted Label: Business

News: The Race is On: Second Private Team Sets Launch Date for Human Spaceflight (SPACE.com) SPACE.com - TORONTO, Canada -- A second\team of rocketeers competing for the  #36;10 million Ansari X Prize, a contest for\privately funded suborbital space flight, has officially announced the first\launch date for its manned rocket.
True Label: Science/Technology
Predicted Label: Science/Technology

News: Ky. Company Wins Grant to Study Peptides (AP) AP - A company founded by a chemistry researcher at the University of Louisville won a grant to develop a method of producing better peptides, which are short chains of amino acids, the building blocks of proteins.
True Label: Science/Technology
Predicted Label: Science/Technology

News: Prediction Unit Helps Forecast Wildfires (AP) 